In [83]:
import pandas as pd
import re
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup
from email import message_from_string


In [84]:
data=pd.read_csv("C:/Users/bechi/OneDrive/Bureau/email_origin.csv")

In [85]:

def clean_email(email_str):
    msg = message_from_string(email_str)
    
    from_address = msg.get('From')
    to_address = msg.get('To')
    subject = msg.get('Subject')
    date = msg.get('Date')
    body = ""   
    if msg.is_multipart():
        for part in msg.get_payload():
            if part.get_content_type() == 'text/html':
                html_content = part.get_payload()
                soup = BeautifulSoup(html_content, "html.parser")
                body = soup.get_text(separator=' ')
                break
    else:
        body = msg.get_payload()
    
    return from_address, to_address, subject, date, body

In [86]:
cleaned_data = pd.DataFrame([clean_email(data["origin"][i]) for i in range(len(data))], 
                            columns=['from_address', 'to_address', 'subject', 'date', 'body'])

C:\Users\bechi\AppData\Local\Temp\ipykernel_5392\1626177188.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, "html.parser")


In [87]:
cleaned_data.isnull().sum()

from_address      0
to_address      554
subject         173
date             25
body              0
dtype: int64

In [88]:
cleaned_data = cleaned_data.dropna()
cleaned_data=cleaned_data[cleaned_data["body"]!=""]

In [89]:
cleaned_data.head()

,from_address,to_address,subject,date,body
0,"""Tomas Jacobs"" <RickyAmes@aol.com>",the00@speedy.uwaterloo.ca,"Generic Cialis, branded quality@","Sun, 08 Apr 2007 21:00:48 +0300",\n \n \n \n \n \n \nDo you feel the pressure t...
1,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,Typo in /debian/README,"Sun, 08 Apr 2007 12:52:30 -0400","Hi, i've just updated from the gulus and I che..."
2,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...",<the00@plg.uwaterloo.ca>,authentic viagra,"Sun, 8 Apr 2007 17:12:19 +0000",authentic viagra \n \nMega authentic V I A G ...
3,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>",opt4@speedy.uwaterloo.ca,Nice talking with ya,"Sun, 08 Apr 2007 17:15:47 -0100","\nHey Billy, \n\nit was really fun going out t..."
6,"""B.J. Badisang"" <mitigatemicroprocessor@g-rich...",<theorize@plg.uwaterloo.ca>,For Theorize,"Sun, 8 Apr 2007 17:19:08 +0480",\n \n \n \n \n \n \n Glad to see you! \n Look ...


In [90]:
duplicated_rows = cleaned_data.duplicated()
print(duplicated_rows.sum())
cleaned_data = cleaned_data.drop_duplicates()

507


In [91]:
def extract_email(email_str):
   left=''
   right=''
   email='@'
   if ('@' in email_str):
         left, right = email_str.split('@', 1)    
         for i in right:
                        if i.isalnum() or i=='.':
                            email+=i
                        else:
                            break
         for i in left[::-1]:
                        if i.isalnum() or i=='.':
                            email=i+email
                        else:
                            break      
    
   return email  


In [92]:
cleaned_data['from_address'] = cleaned_data['from_address'].apply(extract_email)
cleaned_data['to_address'] = cleaned_data['to_address'].apply(extract_email)

In [93]:
print(cleaned_data[(cleaned_data['from_address'] == '@') | (cleaned_data['to_address'] == '@')].size)
cleaned_data = cleaned_data[(cleaned_data['from_address'] != '@') & (cleaned_data['to_address'] != '@')]
print(cleaned_data.size)
cleaned_data["date"][0].split(" ")

2115
315210


['Sun,', '08', 'Apr', '2007', '21:00:48', '+0300']

In [94]:
print(cleaned_data[cleaned_data['date'].str[3] == ','].size)
cleaned_data = cleaned_data[cleaned_data['date'].str[3] == ',']
print(len(cleaned_data))

306885
61377


In [101]:
def months():
    month=[]
    a=[]
    count=0
    for date in cleaned_data['date']:
        day, subdate = date.split(',')
        if day=='':count += count
        subdate = subdate.split(' ')
        if(len(subdate)>6) |(len(subdate)<5):count += count
        if(len(subdate)==5)|(len(subdate)==6):
            if not subdate[2].isalpha():
               a.append(date)
            if(subdate[2]) not in month:
              month.append(subdate[2])
    print(month)  
    print(count)  
    return a

In [102]:
a=months()
print(a)

['Apr', 'Mar', 'Feb', 'May', 'Jan', 'Oct', 'Jun', 'Dec', 'Jul', 'Sep', 'Nov', 'Aug']
0
[]


In [97]:
cleaned_data = cleaned_data[cleaned_data["date"].apply(lambda x: x.split(',')[1].split(' ')[2] in ['Apr', 'Mar', 'Feb', 'May', 'Jan', 'Oct', 'Jun', 'Dec', 'Jul', 'Sep', 'Nov', 'Aug'])]
print(len(cleaned_data))

60705


In [1]:
print(cleaned_data[cleaned_data['date'].apply(lambda x: len(x.split(',')[1].split(' ')) == 6)].sum())

NameError: name 'cleaned_data' is not defined